<a href="https://colab.research.google.com/github/LeeSeungYun1020/Introduction_To_Data_Science/blob/master/classroom/%EC%9D%B4%EC%8A%B9%EC%9C%A4_HW_11_1_Spam_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 11-1. Simple Naive Bayes Classifier


## T1. Load a dataset

The following code loads a dataset consisting of text messages and spam-ham labels.

You can write your own code below the **"TODOs"** to answer the questions.



### Questions:
* Number of spam messges? [*747*]
* Number of ham messages? [*4825*]

In [ ]:
from typing import List, Tuple, Dict, Iterable, Set
from collections import defaultdict
import re
import math
import pandas as pd

url = 'https://raw.githubusercontent.com/mlee-pnu/IDS/main/spam_dataset.csv'
df = pd.read_csv(url)

# TODOs
df.head(10)
numberOfSpam = len(df[df['Category']=='spam'])
numberOfHam = len(df[df['Category']=='ham'])
print("total:", numberOfSpam + numberOfHam)
print("spam:", numberOfSpam)
print("ham:", numberOfHam)

total: 5572
spam: 747
ham: 4825


## T2. Spam filter for individual words

We first defined a function ***tokenize()*** to convert a given text into a set of words. 

Using the function, we now try to count the frequency of each word in each class (spam and ham).

Complete the following code and answer the following questions:
 



### Qeustions: 
*   Number of spam messages containing the word "free": [*170*]

Let's assume we only care for the word "free" to determine if a message is a spam or not. Answer the following questions:

*   P ( *ham* | *free* ) = [*0.2576419213973799*]
*   Is this message spam? [*Yes*]

*Note: Do not apply a smoothing method here.*



In [ ]:
def tokenize(text: str) -> Set[str]:
    text = text.lower()                         
    all_words = re.findall("[a-z0-9']+", text)  
    return set(all_words)                       

In [ ]:
tokens: Set[str] = set()
token_spam_counts: Dict[str, int] = defaultdict(int)
token_ham_counts: Dict[str, int] = defaultdict(int)

spam = df[df.Category == 'spam']
ham = df[df.Category == 'ham']

for msg in spam['Message'].to_list():
  for token in tokenize(msg):
    tokens.add(token)
    token_spam_counts[token] += 1

for msg in ham['Message'].to_list():
  for token in tokenize(msg):
    tokens.add(token)
    token_ham_counts[token] += 1

# TODOs
word = "free"
n_word_spam = token_spam_counts[word]   # frequency of the word in spam messages
n_word_ham =  token_ham_counts[word]    # frequency of the word in ham messages
print(f"Number of '{word}' in spam msg", n_word_spam)
print(f"Number of '{word}' in ham msg", n_word_ham)

p_spam = numberOfSpam / (numberOfSpam + numberOfHam)      # P(spam)
p_ham = numberOfHam / (numberOfSpam + numberOfHam)        # P(ham)
p_word_given_spam = n_word_spam / numberOfSpam  # P(word|spam)
p_word_given_ham = n_word_ham / numberOfHam     # P(word|ham)
print("P(spam) =", p_spam)
print("P(ham) =", p_ham)
print(f"P('{word}'|spam) =", p_word_given_spam)
print(f"P('{word}'|ham) =", p_word_given_ham)

p_word = (n_word_spam + n_word_ham) / len(df)
print(f"P({'word'}) =", p_word)

# P(spam|word)
p_spam_given_word = p_spam * p_word_given_spam / p_word
# P(ham|word)
p_ham_given_word = p_ham * p_word_given_ham / p_word
print(f"P(spam|'{word}') =", p_spam_given_word)
print(f"P(ham|'{word}') =", p_ham_given_word)

print("----------")
print("Number of spam messages containing 'free':", n_word_spam)
print(f"P(ham|'{word}') = {p_ham_given_word}")

Number of 'free' in spam msg 170
Number of 'free' in ham msg 59
P(spam) = 0.13406317300789664
P(ham) = 0.8659368269921034
P('free'|spam) = 0.22757697456492637
P('free'|ham) = 0.0122279792746114
P(word) = 0.04109834888729361
P(spam|'free') = 0.74235807860262
P(ham|'free') = 0.2576419213973799
----------
Number of spam messages containing 'free': 170
P(ham|'free') = 0.2576419213973799


## T3. Spam filter that combines words: Naive Bayes

You received a text message "just do it" from an unknown sender.

Complete the function ***predict()*** that outputs the probability of the message being spam and the predicted label of the message. 


### Questions:

*   P ( *spam* | *text* ) = [*3.3152855896268886e-06*]
*   Is this text message spam? [*No*]

*Note: You do not need to apply a smoothing method here.*



In [ ]:
text = "just do it"
#text = "Call to your phone"
#text = "Free mobile text now"
#text = "I and you"
#text = "call to me"

# TODOs
def predict(text: str):
  text_tokens = tokenize(text)
  log_spam = 0.0
  log_ham = 0.0

  for token in tokens:
    pt_spam = token_spam_counts[token] / numberOfSpam
    pt_ham = token_ham_counts[token] / numberOfHam
    if token in text_tokens:
      log_spam += math.log(pt_spam)
      log_ham += math.log(pt_ham)
    else:
      log_spam += math.log(1.0 - pt_spam)
      log_ham += math.log(1.0 - pt_ham)
  p_spam = math.exp(log_spam)
  p_ham = math.exp(log_ham)
  label = "ham"
  if p_spam > p_ham:
    label = "spam"
  prob = p_spam / (p_spam + p_ham)
  return prob, label

print(predict(text))

(3.3152855896268886e-06, 'ham')


## T4. Smoothing method

You again received two text messages from unknown senders.

Complete the function ***spamFilter()*** that classifies a given message. 

You may want to apply a smoothing method for this task.


### Questions:

*   Is textA spam?: [*Yes*]
*   Is textB spam?: [*No*]


In [ ]:
textA = "reward! download your free ticket from our website www.pnu.edu"
textB = "call me and get your money back"

# TODOs
def spamFilter(text: str):
  k = 0.5
  text_tokens = tokenize(text)
  log_spam = 0.0
  log_ham = 0.0

  for token in tokens:
    pt_spam = (token_spam_counts[token] + k) / (numberOfSpam + 2 * k)
    pt_ham = (token_ham_counts[token] + k) / (numberOfHam + 2 * k)
    if token in text_tokens:
      log_spam += math.log(pt_spam)
      log_ham += math.log(pt_ham)
    else:
      log_spam += math.log(1.0 - pt_spam)
      log_ham += math.log(1.0 - pt_ham)
  p_spam = math.exp(log_spam)
  p_ham = math.exp(log_ham)
  label = "ham"
  if p_spam > p_ham:
    label = "spam"
  # prob = p_spam / (p_spam + p_ham)
  # print(prob)
  return label

print(spamFilter(textA))
print(spamFilter(textB))


spam
ham


In [ ]:
from collections import Counter
predictions = [(category, spamFilter(message)) for category, message in zip(df['Category'], df['Message'])]

In [ ]:
confusion_matrix = Counter((category == 'spam', is_spam == 'spam') for category, is_spam in predictions)
print(confusion_matrix)

Counter({(False, False): 4819, (True, True): 722, (True, False): 25, (False, True): 6})
